In [ ]:
import datetime
import sys
import os
import torch
import numpy as np
import random
from pathlib import Path
from collections import namedtuple
from miditok import REMIPlus, MMM
from miditok.constants import ADDITIONAL_TOKENS
from torch.nn import functional as F
from importlib import reload

In [ ]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CTX_LEN = 512
PRECISION = 'bf16'

os.environ['RWKV_T_MAX'] = str(CTX_LEN)
os.environ['RWKV_FLOAT_MODE'] = PRECISION

sys.path.append('./src/model')
sys.path.append('./src/tools')

os.getcwd()

In [ ]:
from runner import RWKV_RNN

In [ ]:
PROJ_NAME = 'sequences'
IS_BPE = True
TOKENS_PATH = f"/home/nico/data/ai/models/midi/{PROJ_NAME}{'/bpe' if IS_BPE else ''}"

Path(f'./out/{PROJ_NAME}').mkdir(parents=True, exist_ok=True)

BINS_VELOCITY = (24)
BINS_TEMPO = (24)

additional_tokens = ADDITIONAL_TOKENS
additional_tokens['Chord'] = True
additional_tokens['TimeSignature'] = True
additional_tokens['Program'] = True
additional_tokens['nb_tempos'] = BINS_TEMPO
TOKENIZER = MMM(
    additional_tokens=additional_tokens, 
    params=f'{TOKENS_PATH}/token_params.cfg',
    nb_velocities=BINS_VELOCITY
)

ORIG_VOCAB_SIZE = len(TOKENIZER.vocab)
BPE_VOCAB_SIZE = int(ORIG_VOCAB_SIZE * 1.25)

(ORIG_VOCAB_SIZE, BPE_VOCAB_SIZE, len(TOKENIZER))

In [ ]:
N_EMBED = 768
N_LAYER = 10
CTX_LEN = 512

params = {
    'ctx_len': CTX_LEN,
    'n_embd': N_EMBED,
    'n_layer': N_LAYER,
}

params_obj = namedtuple('RWKVParams', params.keys())(*params.values())

In [ ]:
import matplotlib.pyplot as plt

MAX_ITER = 1024*3

# this is where we introduce some randomness
NOISE_LEVEL = 0.55
NOISE_FREQ = 10
PHASE = 0


def gen_sin_wave(total_iterations, min_value, max_value, noise_scale, noise_frequency, main_phase):
    """
    Generate a sinusoidal wave with optional noise.

    Args:
        total_iterations (int): The total number of iterations.
        min_value (float): The minimum value of the wave.
        max_value (float): The maximum value of the wave.
        noise_scale (float): The scale factor for the noise.
        noise_frequency (float): The frequency of the noise wave.
        main_phase (float): The phase of the main wave.

    Returns:
        list: A list of generated wave values.

    """
    progress = np.linspace(0, 1, total_iterations)
    main_wave = np.sin(2 * np.pi * progress + main_phase)
    noise_wave = np.sin(2 * np.pi * noise_frequency * progress - main_phase / 2)
    noise = noise_scale * noise_wave
    values = min_value + (max_value - min_value) * \
        (1 + main_wave) / 2 + noise

    np.clip(values, min_value, max_value, out=values)

    return values.tolist()


temp_values = gen_sin_wave(MAX_ITER, 0.25, 0.85, NOISE_LEVEL, NOISE_FREQ, PHASE)
top_p_values = gen_sin_wave(MAX_ITER, 0.75, 1.0, NOISE_LEVEL, NOISE_FREQ*2, PHASE+6)

plt.plot(temp_values)
plt.plot(top_p_values)

In [ ]:
import types

SEED = random.randint(1000, 10000)
np.random.seed(SEED)

args = types.SimpleNamespace()
args.RUN_DEVICE = "cuda"
args.FLOAT_MODE = PRECISION
args.map_location = 'cpu'
args.base_model = f'/home/nico/dev/projects/ai/musai/out/{PROJ_NAME}/rwkv-40'
args.n_layer = params['n_layer']
args.n_embd = params['n_embd']
args.ctx_len = int(params['ctx_len'])

model_rnn = RWKV_RNN(args)
model_rnn.to(DEVICE)

In [ ]:
import json
from miditoolkit import MidiFile

init_state = None
out_tokens = []
tokens_file_paths = list(Path(TOKENS_PATH).glob('*.json'))

random.shuffle(tokens_file_paths)

SAMPLE_TOKENS_FILE = '/home/nico/data/ai/models/midi/sequences/bpe/133_beethoven_sonate_op2_scherzo_mid.json'

# token_ids = json.load(open(tokens_file_paths[0]))['ids']
token_ids = json.load(open(SAMPLE_TOKENS_FILE))['ids']
# sample = TOKENIZER.midi_to_tokens(MidiFile('examples/2023-06-11T12-59-05-739764.mid'))
# token_ids = sample.ids
max_seq = 256
init_tokens = token_ids[:max_seq]


In [ ]:
import re
from runner import sample_logits, repetition_penalty

d = datetime.datetime.now().isoformat()
d = re.sub(r'[^\dT]{1,}', '-', d)

for x in range(len(init_tokens)):
    init_out, init_state = model_rnn.forward([init_tokens[x]], init_state)

    # ignore padding
    init_out[0] = -float('inf')

    out_token = sample_logits(init_out.detach(), temperature=0.15)

    out_tokens.append(out_token)

for i in range(MAX_ITER):
    out, init_state = model_rnn.forward(out_tokens, init_state)

    # ignore padding
    out[0] = -float('inf')

    # apply repetition penalty
    out = repetition_penalty(out, out_tokens, [
                             4, 5, 6, 7], repetition_penalty=1.25, seq_len=128, decay_factor=0.8)

    out_token = sample_logits(
        out.detach(), temperature=temp_values[i], top_p=top_p_values[i])

    out_tokens.append(out_token)


In [ ]:
from tokenizer import parse_bpe_tokens

seqin = parse_bpe_tokens(TOKENIZER, token_ids)
seqout = parse_bpe_tokens(TOKENIZER, out_tokens)
tailhead_len = max_seq

set(seqin[:tailhead_len]) - set(seqout[:tailhead_len])


In [ ]:
', '.join(seqin[:tailhead_len] + ['...'] + seqin[len(seqin)-tailhead_len:])

In [ ]:
', '.join(seqout[:tailhead_len] + ['...'] + seqout[len(seqout)-tailhead_len:])

In [ ]:
fname = f'out/{PROJ_NAME}/{d}.mid'
fname_orig = f'out/{PROJ_NAME}/{d}_orig.mid'

TOKENIZER(out_tokens).dump(fname)
TOKENIZER(token_ids).dump(fname_orig)

[fname, fname_orig]